In [1]:
# import all necessary modules and set parameters
from __future__ import print_function, absolute_import, division
# load modules and cythonmagic
import numpy as np
from time import time
%load_ext Cython

In [2]:
# Define class Data. It will save all the necessary data for the problem
# Example is about computing interaction between particles with given matrix of interactions
class Data(object):
    """This is container for data for the problem."""
    """First of all, it requires methods check_far, compute_aux, divide and __len__."""
    def __init__(self, particles, matrix):
        """save particles and matrix"""
        self.particles = particles
        self.matrix = matrix
        # main requirement here is to set self.count -- number of particles and self.dim -- dimensionality of the problem
        self.count, self.dim = particles.shape
        
    """All other functions must have exactly the same parameters, as required."""
    
    def check_far(self, bb0, bb1):
        """checks if bounding boxes bb0 and bb1 do not cross each other."""
        mean0 = bb0.mean(axis = 1)
        mean1 = bb1.mean(axis = 1)
        dist = np.linalg.norm(mean1-mean0)
        diag = max(np.linalg.norm(bb0[:,1]-bb0[:,0]), np.linalg.norm(bb1[:,1]-bb1[:,0]))
        return dist > diag
    
    def compute_aux(self, index):
        """computes bounding boxes, requires self.particles defined."""
        selected = self.particles[index]
        return np.hstack([selected.min(axis = 0).reshape(3,1), selected.max(axis = 0).reshape(3,1)])

    def divide(self, index):
        """divides cluster into subclusters."""
        vertex = self.particles[index].copy()
        center = vertex.mean(axis = 0)
        vertex -= center.reshape(1, self.dim)
        normal = np.linalg.svd(vertex, full_matrices = 0)[2][0]
        scal_dot = normal.dot(vertex.T)
        permute = scal_dot.argsort()
        scal_dot = scal_dot[permute]
        k = scal_dot.searchsorted(0)
        return permute, [0, k, permute.size]

    def __len__(self):
        return self.count

In [3]:
%%cython
import numpy as np
# this function generates matrix with elements: a[i,j] = 1/r(x[i], y[j]), where r(x,y) is a distance between points in 3d space
def gen_matrix(x, y):
    a = np.ndarray((x.shape[0], y.shape[0]))
    cdef double [:,:] a_buf = a
    cdef double [:,:] x_buf = x
    cdef double [:,:] y_buf = y
    cdef double z[3]
    cdef double tmp
    cdef int i, j
    for i in range(x_buf.shape[0]):
        for j in range(y_buf.shape[0]):
            z[0] = x_buf[i,0]-y_buf[j,0]
            z[1] = x_buf[i,1]-y_buf[j,1]
            z[2] = x_buf[i,2]-y_buf[j,2]
            tmp = z[0]*z[0]+z[1]*z[1]+z[2]*z[2]
            if tmp < 1e-30:
                a_buf[i, j] = 0.0
            else:
                a_buf[i, j] = tmp**(-0.5)
    return a

In [4]:
# Here we generate 1000 particles randomly in cube [0;1]^3
np.random.seed(0)
particles = np.random.rand(1000, 3)
# Let matrix of interactions be equal to 1/r
dense_matrix = gen_matrix(particles, particles)

In [5]:
# Generate object of class Data based on particles and matrix of interactions
data = Data(particles, dense_matrix)

# Initialize cluster trees with root node
from h2tools import ClusterTree
block_size = 25
tree = ClusterTree(data, block_size)

# Set function, that returns submatrix of the matrix of interactions
def func(data1, rows, data2, columns):
    return data1.matrix[rows][:,columns]

# Hack to prevent bug of qr decomposition for 130x65 matrices (multithread mkl bug)
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

# Generate block cluster tree in variable `problem`
from h2tools import Problem
symmetric = 1
verbose = True
problem = Problem(func, tree, tree, symmetric, verbose)

Cluster trees are generated in 0.209475040436 seconds
Depth level of each cluster tree: 8
Row cluster tree
    nodes : 115
    leaves: 58
Column cluster tree
    nodes : 115
    leaves: 58


In [6]:
# computing multicharge representation
from __future__ import print_function

from h2tools.mcbh import mcbh

print('Computing MCBH, relative error parameter tau set to 1e-4')
h2matrix = mcbh(problem, tau=1e-3, iters=1, onfly=0, verbose=1)
print('memory for uncompressed approximation: '+str(h2matrix.nbytes()/1024./1024)+' MB')
# error measure with pypropack
print('relative error of approximation:', h2matrix.diffnorm(far_only=0))
# compressing approximant
h2matrix.svdcompress(1e-3, verbose=1)
# error measure with pypropack
print('relative error of far-field approximation:', h2matrix.diffnorm(far_only=1))

h2matrix.svdcompress(1e-2, verbose=1)
# error measure with pypropack
print('relative error of far-field approximation:', h2matrix.diffnorm(far_only=1))

Computing MCBH, relative error parameter tau set to 1e-4
Far-field interactions(MCBH method):
    Function calls: 853
    Function values computed: 849838
    Function time, seconds: 0.03
    Average time per function value, seconds: 2.95e-08
    Maxvol time, seconds: 0.13267493248
Near-field interactions:
    Function calls: 501
    Function values computed: 181023
    Function time, seconds: 0.02
    Average time per function value, seconds: 9.39e-08
Total time, seconds: 0.20
Memory:
    Basises, MB: 0.02
    Transfer matrices, MB: 0.13
    Far-field interactions, MB: 0.14
    Near-field interactions, MB: 0.12
Total memory, MB: 0.41
memory for uncompressed approximation: 0.406196594238 MB
relative error of approximation: 0.000413797990161
memory BEFORE SVD-compression: 0.41MB
memory AFTER SVD-compression: 0.39MB
recompression time: 0.162955999374
relative error of far-field approximation: 0.00087821686283
memory BEFORE SVD-compression: 0.39MB
memory AFTER SVD-compression: 0.33MB
reco

In [7]:
print(h2matrix)

H2matrix at 0x105357250
    Structure (h2/mcbh): h2
    Memory layout (low/full): full
    Shape: [1000, 1000]
    Total memory, MB: 0.33
        Basises, MB: 0.00
        Transfer matrices, MB: 0.07
        Far-field interactions, MB: 0.14
        Near-field interactions, MB: 0.12
